#### Snapshot of global MCS, BL measure, and CWV

In [ ]:
import os
import sys
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from datetime import datetime
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from scipy.stats import linregress

import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
cwv_data = xr.open_dataset('/neelin2020/ERA-5_cwv_That/era5_cwv_that_2002_01_01.nc')
thetae_data = xr.open_dataset('/neelin2020/ERA-5_buoy/layer_thetae/era5_2layers_thetae_2002_01_01.nc')
mcs_data = xr.open_dataset('/neelin2020/mcs_flextrkr/20020101.0000_20030101.0000/mcstrack_20020101_0030.nc')

In [ ]:
mcs_sub = mcs_data.cloudtracknumber_nomergesplit.sel(lat=slice(-30,30)).isel(time=0)
lon_reset = mcs_sub.lon
lon_reset = lon_reset.where(lon_reset > 0, 360+lon_reset) # converting lon as 0 to 359.75
mcs_sub.coords['lon'] = lon_reset # converting lon as -180 to 180
mcs_sub = mcs_sub.sortby('lon')
mcs_sub = mcs_sub.where(mcs_sub > 0, 0)
mcs_sub = mcs_sub.where(mcs_sub == 0, 1) # create mcs binary masks
        
cwv_sub = cwv_data.cwv.sel(time=datetime(2002,1,1,0), lat=slice(-30,30))

# calculate buoyancy estimate
thetae_sub = thetae_data.sel(time=datetime(2002,1,1,0), lat=slice(-30,30))
thetae_bl = thetae_sub.thetae_bl
thetae_sat_lt = thetae_sub.thetae_sat_lt
thetae_lt = thetae_sub.thetae_lt
wb = 0.4
wl = 0.6
Buoy_CAPE = (9.81/(340*3)) * wb * ((thetae_bl-thetae_sat_lt)/thetae_sat_lt) * 340
Buoy_SUBSAT = (9.81/(340*3))* wl * ((thetae_sat_lt-thetae_lt)/thetae_sat_lt) * 340
Buoy_TOT = Buoy_CAPE - Buoy_SUBSAT

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,3),subplot_kw={'projection': ccrs.PlateCarree()})

ax.coastlines(color='k',linewidth=0.7)
ax.add_feature(cfeat.LAND,zorder=3,edgecolor='grey',facecolor='none',linewidth=0.3) # maskout LAND
ax.add_feature(cfeat.BORDERS,zorder=3,edgecolor='grey',facecolor='none',linewidth=0.3)
ax.add_feature(cfeat.STATES,zorder=3,edgecolor='grey',facecolor='none',linewidth=0.3) 

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='k', alpha=0.4, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 9, 'color': 'k'}
gl.ylabel_style = {'size': 9, 'color': 'k'}

# BL measures (K)
cp = ax.pcolormesh(Buoy_TOT.lon, Buoy_TOT.lat, 340*3/9.8*Buoy_TOT, vmin=-15, vmax=2, cmap='CMRmap_r')
cp.cmap.set_under('w')
cbar = plt.colorbar(cp, ax=ax, shrink=0.4, pad=0.02, extend='min')
cbar.set_label('B$_{L}$ (K)', fontsize=10)
cbar.set_ticks([-15,-10,-5,0])
ax.contour(Buoy_TOT.lon, Buoy_TOT.lat, 340*3/9.8*Buoy_TOT, levels=[-10], colors=['k'],
          linewidths=1)
# CWV 
ax.contour(cwv_sub.lon, cwv_sub.lat, cwv_sub, levels=[48], colors=['w'],linewidths=1)
# tracked MCSs contour
ax.contour(mcs_sub.lon, mcs_sub.lat, mcs_sub, levels=[0,1], colors=['g'], zorder=2,
          linewidths=1)

ax.set_title('2002-01-01, 00:30Z, FLEXTRKR-MCS (green) / B$_{L}$ / CWV (white)', fontsize=10)

In [ ]:
fig.savefig('/scratch/wmtsai/temp_mcs/fig/MCS_BL_CWV_map_tropics.pdf', bbox_inches='tight')